<a href="https://colab.research.google.com/github/blauveltmr/Data-Analysis/blob/master/Parameters_All_Strategy_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
start_date = '2015-01-01'
end_date = '2021-02-17'
date_fmt = '%Y-%m-%d'

init_balance = 15000.00

!pip install yfinance
!pip install ta
import yfinance as yf
import ta
import pandas as pd
from datetime import date, timedelta, datetime
from IPython.display import clear_output
import time

tic = time.time()

from google.colab import drive
drive.flush_and_unmount
drive.mount('/content/drive', force_remount=True)

#Compute Buffer
start_date_buffer = datetime.strptime(start_date, date_fmt) - timedelta(days=365)
start_date_buffer = start_date_buffer.strftime(date_fmt)
start_date_buffer

In [ ]:
def get_stock_backtest_data(ticker, start_date, end_date):
  date_fmt = '%Y-%m-%d'

  start_date_buffer = datetime.strptime(start_date, date_fmt) - timedelta(days=365)
  start_date_buffer = start_date_buffer.strftime(date_fmt)

  df = yf.download(ticker, start=start_date_buffer, end=end_date)

  return df

In [ ]:
class StockBacktestData:
  def __init__(self, ticker, start_date, end_date):
    self._ticker = ticker
    self._backtest_start_buffer_days = 365
    self._buffer_days = 90

    init_start_date, init_end_date = self._get_buffer_start_end_dates(start_date, end_date)
    self._data = self._download_stock_backtest_data(self._ticker, init_start_date, init_end_date)

  
  def _get_buffer_start_end_dates(self, start_date, end_date):
    date_fmt = '%Y-%m-%d'
    init_start_date = datetime.strptime(start_date, date_fmt) - timedelta(
        days=(self._backtest_start_buffer_days + self._buffer_days)
        )
    
    init_start_date = init_start_date.strftime(date_fmt)

    init_end_date = datetime.strptime(end_date, date_fmt) + timedelta(days=self._buffer_days)

    if init_end_date > datetime.today():
      init_end_date = datetime.today()

    init_end_date = init_end_date.strftime(date_fmt)

    return init_start_date, init_end_date


  def _get_backtest_start_date(self, start_date):
    date_fmt = '%Y-%m-%d'
    start_date_buffer = datetime.strptime(start_date, date_fmt) - timedelta(
        days=self._backtest_start_buffer_days
        )
    
    start_date_buffer = start_date_buffer.strftime(date_fmt)
    return start_date_buffer


  def _download_stock_backtest_data(self, ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df


  def get_stock_backtest_data(self, start_date, end_date):
    start_date_buffer = self._get_backtest_start_date(start_date)
    df = self._data[(self._data.index >= start_date_buffer) & (self._data.index <= end_date)]
    return df.copy()

**Strategies**

In [ ]:
def strategy_KeltnerChannel_origin(df, **kwargs):
  n = kwargs.get('n', 10)
  data = df.copy()

  k_band = ta.volatility.KeltnerChannel(data.High, data.Low, data.Close, n)

  data['K_BAND_UB'] = k_band.keltner_channel_hband().round(4)
  data['K_BAND_LB'] = k_band.keltner_channel_lband().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)
  
  data['LONG'] = (data.Close <= data.K_BAND_LB) & (data.CLOSE_PREV > data.K_BAND_LB)
  data['EXIT_LONG'] = (data.Close >= data.K_BAND_UB) & (data.CLOSE_PREV < data.K_BAND_UB)

  data['SHORT'] = (data.Close >= data.K_BAND_UB) & (data.CLOSE_PREV < data.K_BAND_UB)
  data['EXIT_SHORT'] = (data.Close <= data.K_BAND_LB) & (data.CLOSE_PREV > data.K_BAND_LB)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_KeltnerChannel_origin_long(df, **kwargs):
  n = kwargs.get('n', 10)
  data = df.copy()

  k_band = ta.volatility.KeltnerChannel(data.High, data.Low, data.Close, n)

  data['K_BAND_UB'] = k_band.keltner_channel_hband().round(4)
  data['K_BAND_LB'] = k_band.keltner_channel_lband().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)
  
  data['LONG'] = (data.Close <= data.K_BAND_LB) & (data.CLOSE_PREV > data.K_BAND_LB)
  data['EXIT_LONG'] = (data.Close >= data.K_BAND_UB) & (data.CLOSE_PREV < data.K_BAND_UB)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_BollingerBands(df, **kwargs):
  n = kwargs.get('n', 10)
  n_rng = kwargs.get('n_rng', 2)
  data = df.copy()

  boll = ta.volatility.BollingerBands(data.Close, n, n_rng)

  data['BOLL_LBAND_INDI'] = boll.bollinger_lband_indicator()
  data['BOLL_UBAND_INDI'] = boll.bollinger_hband_indicator()

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = data.BOLL_LBAND_INDI == 1
  data['EXIT_LONG'] = data.BOLL_UBAND_INDI == 1

  data['SHORT'] = data.BOLL_UBAND_INDI == 1
  data['EXIT_SHORT'] = data.BOLL_LBAND_INDI == 1

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_BollingerBands_long(df, **kwargs):
  n = kwargs.get('n', 10)
  n_rng = kwargs.get('n_rng', 2)
  data = df.copy()
  
  boll = ta.volatility.BollingerBands(data.Close, n, n_rng)

  data['BOLL_LBAND_INDI'] = boll.bollinger_lband_indicator()
  data['BOLL_UBAND_INDI'] = boll.bollinger_hband_indicator()

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = data.BOLL_LBAND_INDI == 1
  data['EXIT_LONG'] = data.BOLL_UBAND_INDI == 1

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

# df = get_stock_backtest_data(ticker, '2019-01-01', '2019-12-31')
# strategy_BollingerBands(df, n=10, n_rng=2)

In [ ]:
def strategy_MA(df, **kwargs):
  n = kwargs.get('n', 50)
  ma_type = kwargs.get('ma_type', 'sma')
  ma_type = ma_type.strip().lower()
  data = df.copy()
  
  if ma_type == 'sma':
    sma = ta.trend.SMAIndicator(data.Close, n)
    data['MA'] = sma.sma_indicator().round(4)
  elif ma_type == 'ema':
    ema = ta.trend.EMAIndicator(data.Close, n)
    data['MA'] = ema.ema_indicator().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = (data.Close > data.MA) & (data.CLOSE_PREV <= data.MA)
  data['EXIT_LONG'] = (data.Close < data.MA) & (data.CLOSE_PREV >= data.MA)

  data['SHORT'] = (data.Close < data.MA) & (data.CLOSE_PREV >= data.MA)
  data['EXIT_SHORT'] = (data.Close > data.MA) & (data.CLOSE_PREV <= data.MA)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)
  return data

def strategy_MACrossover(df, **kwargs):
  n_slow = kwargs.get('n_slow', 13)
  n_fast = kwargs.get('n_fast', 5)
  n_middle = kwargs.get('n_middle', 8)
  data = df.copy()
  
  
  sma_fast = ta.trend.SMAIndicator(data.Close, n_fast)
  data['SMA_FAST'] = sma_fast.sma_indicator().round(4)
  sma_slow = ta.trend.SMAIndicator(data.Close, n_slow)
  data['SMA_SLOW'] = sma_slow.sma_indicator().round(4)
  sma_middle = ta.trend.SMAIndicator(data.Close, n_middle)
  data['SMA_MIDDLE'] = sma_middle.sma_indicator().round(4)

  data['SMA_FAST_PREV'] = data.SMA_FAST.shift(1)
  data['SMA_MIDDLE_PREV'] = data.SMA_MIDDLE.shift(1)
  data['SMA_SLOW_PREV'] = data.SMA_SLOW.shift(1)

  data['LONG'] = (data.SMA_FAST > data.SMA_MIDDLE) & (data.SMA_FAST > data.SMA_SLOW) & (data.SMA_MIDDLE > data.SMA_SLOW) & ((data.SMA_FAST_PREV < data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV < data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV < data.SMA_SLOW_PREV))
  data['EXIT_LONG'] = (data.SMA_FAST < data.SMA_MIDDLE) & (data.SMA_FAST < data.SMA_SLOW) & (data.SMA_MIDDLE < data.SMA_SLOW) & ((data.SMA_FAST_PREV > data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV > data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV > data.SMA_SLOW_PREV))

  data['SHORT'] = (data.SMA_FAST < data.SMA_MIDDLE) & (data.SMA_FAST < data.SMA_SLOW) & (data.SMA_MIDDLE < data.SMA_SLOW) & ((data.SMA_FAST_PREV > data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV > data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV > data.SMA_SLOW_PREV))
  data['EXIT_SHORT'] = (data.SMA_FAST > data.SMA_MIDDLE) & (data.SMA_FAST > data.SMA_SLOW) & (data.SMA_MIDDLE > data.SMA_SLOW) & ((data.SMA_FAST_PREV < data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV < data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV < data.SMA_SLOW_PREV))

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

def strategy_MA_long(df, **kwargs):
  n = kwargs.get('n', 50)
  ma_type = kwargs.get('ma_type', 'sma')
  ma_type = ma_type.strip().lower()
  data = df.copy()
  
  if ma_type == 'sma':
    sma = ta.trend.SMAIndicator(data.Close, n)
    data['MA'] = sma.sma_indicator().round(4)
  elif ma_type == 'ema':
    ema = ta.trend.EMAIndicator(data.Close, n)
    data['MA'] = ema.ema_indicator().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = (data.Close > data.MA) & (data.CLOSE_PREV <= data.MA)
  data['EXIT_LONG'] = (data.Close < data.MA) & (data.CLOSE_PREV >= data.MA)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


# df = get_stock_backtest_data(ticker, '2019-01-01', '2019-12-31')
# strategy_SMA(df, n=10, ma_type='ema')

In [ ]:
def strategy_MACD(df, **kwargs):
  n_slow = kwargs.get('n_slow', 26)
  n_fast = kwargs.get('n_fast', 12)
  n_sign = kwargs.get('n_sign', 9)
  data = df.copy()

  macd = ta.trend.MACD(data.Close, n_slow, n_fast, n_sign)

  data['MACD_DIFF'] = macd.macd_diff().round(4)
  data['MACD_DIFF_PREV'] = data.MACD_DIFF.shift(1)

  data['LONG'] = (data.MACD_DIFF > 0) & (data.MACD_DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.MACD_DIFF < 0) & (data.MACD_DIFF_PREV >= 0)

  data['SHORT'] = (data.MACD_DIFF < 0) & (data.MACD_DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.MACD_DIFF > 0) & (data.MACD_DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_MACD_long(df, **kwargs):
  n_slow = kwargs.get('n_slow', 26)
  n_fast = kwargs.get('n_fast', 12)
  n_sign = kwargs.get('n_sign', 9)
  data = df.copy()

  macd = ta.trend.MACD(data.Close, n_slow, n_fast, n_sign)

  data['MACD_DIFF'] = macd.macd_diff().round(4)
  data['MACD_DIFF_PREV'] = data.MACD_DIFF.shift(1)

  data['LONG'] = (data.MACD_DIFF > 0) & (data.MACD_DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.MACD_DIFF < 0) & (data.MACD_DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

# df = get_stock_backtest_data(ticker, '2019-01-01', '2019-12-31')
# strategy_MACD(df, n_slow=26, n_fast=12, n_sign=9)

In [ ]:
def strategy_RSI(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  rsi = ta.momentum.RSIIndicator(data.Close, n)

  data['RSI'] = rsi.rsi().round(4)
  data['RSI_PREV'] = data.RSI.shift(1)

  data['LONG'] = (data.RSI > 30) & (data.RSI_PREV <= 30)
  data['EXIT_LONG'] = (data.RSI < 70) & (data.RSI_PREV >= 70)

  data['SHORT'] = (data.RSI < 70) & (data.RSI_PREV >= 70)
  data['EXIT_SHORT'] = (data.RSI > 30) & (data.RSI_PREV <= 30)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_RSI_long(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  rsi = ta.momentum.RSIIndicator(data.Close, n)

  data['RSI'] = rsi.rsi().round(4)
  data['RSI_PREV'] = data.RSI.shift(1)

  data['LONG'] = (data.RSI > 30) & (data.RSI_PREV <= 30)
  data['EXIT_LONG'] = (data.RSI < 70) & (data.RSI_PREV >= 70)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

# df = get_stock_backtest_data(ticker, '2019-01-01', '2019-12-31')
# strategy_RSI(df, n_slow=26, n_fast=12, n_sign=9)

In [ ]:
def strategy_WR(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  wr = ta.momentum.WilliamsRIndicator(data.High, data.Low, data.Close, n)

  data['WR'] = wr.williams_r().round(4)
  data['WR_PREV'] = data.WR.shift(1)

  data['LONG'] = (data.WR > -80) & (data.WR_PREV <= -80)
  data['EXIT_LONG'] = (data.WR < -20) & (data.WR_PREV >= -20)

  data['SHORT'] = (data.WR < -20) & (data.WR_PREV >= -20)
  data['EXIT_SHORT'] = (data.WR > -80) & (data.WR_PREV <= -80)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_WR_long(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  wr = ta.momentum.WilliamsRIndicator(data.High, data.Low, data.Close, n)

  data['WR'] = wr.williams_r().round(4)
  data['WR_PREV'] = data.WR.shift(1)

  data['LONG'] = (data.WR > -80) & (data.WR_PREV <= -80)
  data['EXIT_LONG'] = (data.WR < -20) & (data.WR_PREV >= -20)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

# df = get_stock_backtest_data(ticker, '2019-01-01', '2019-12-31')
# strategy_WR(df, n_slow=26, n_fast=12, n_sign=9)

In [ ]:
def strategy_Stochastic_fast(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  data['DIFF'] = data['K'] - data['D']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_Stochastic_fast_long(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  data['DIFF'] = data['K'] - data['D']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

# df = get_stock_backtest_data(ticker, '2019-01-01', '2019-12-31')
# strategy_Stochastic_fast(df, k=20, d=5)

In [ ]:
def strategy_Stochastic_slow(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  dd = kwargs.get('dd', 3)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  
  ma = ta.trend.SMAIndicator(data.D, dd)
  data['DD'] = ma.sma_indicator().round(4)

  data['DIFF'] = data['D'] - data['DD']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_Stochastic_slow_long(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  dd = kwargs.get('dd', 3)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  
  ma = ta.trend.SMAIndicator(data.D, dd)
  data['DD'] = ma.sma_indicator().round(4)

  data['DIFF'] = data['D'] - data['DD']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

# df = get_stock_backtest_data(ticker, '2019-01-01', '2019-12-31')
# strategy_Stochastic_slow(df, k=20, d=5, dd=3)

In [ ]:
def strategy_Ichmoku(df, **kwargs):
  n_conv = kwargs.get('n_conv', 9)
  n_base = kwargs.get('n_base', 26)
  n_span_b = kwargs.get('n_span_b', 26)
  data = df.copy()

  ichmoku = ta.trend.IchimokuIndicator(data.High, data.Low, n_conv, n_base, n_span_b)

  data['BASE'] = ichmoku.ichimoku_base_line().round(4)
  data['CONV'] = ichmoku.ichimoku_conversion_line().round(4)

  data['DIFF'] = data['CONV'] - data['BASE']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_Ichmoku_long(df, **kwargs):
  n_conv = kwargs.get('n_conv', 9)
  n_base = kwargs.get('n_base', 26)
  n_span_b = kwargs.get('n_span_b', 26)
  data = df.copy()

  ichmoku = ta.trend.IchimokuIndicator(data.High, data.Low, n_conv, n_base, n_span_b)

  data['BASE'] = ichmoku.ichimoku_base_line().round(4)
  data['CONV'] = ichmoku.ichimoku_conversion_line().round(4)

  data['DIFF'] = data['CONV'] - data['BASE']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

# df = get_stock_backtest_data('ISF.L', '2019-01-01', '2019-12-31')
# strategy_Ichmoku(df, n_conv=9, n_base=26, n_span_b=26)

In [ ]:
def prepare_stock_ta_backtest_data(df, start_date, end_date, strategy, **strategy_params):
  df_strategy = strategy(df, **strategy_params)
  bt_df = df_strategy[(df_strategy.index >= start_date) & (df_strategy.index <= end_date)]
  return bt_df

In [ ]:
def run_stock_ta_backtest(bt_df, stop_loss_lvl=None):
  balance = init_balance
  market_value = init_balance
  pnl = 0.00
  position = 0
  stopthresh = 'no'
  last_signal = 'hold'
  last_price = 0.00
  c = 0

  trade_date_start = []
  trade_date_end = []
  trade_days = []
  trade_side = []
  trade_pnl = []
  trade_ret = []
  trade_balance = []

  cum_value = []

  for index, row in bt_df.iterrows():
      # check and close any positions
      if row.EXIT_LONG and last_signal == 'long' and position > 0:
        trade_date_end.append(row.name)
        trade_days.append(c)

        pnl = (row.Close - last_price) * position
        trade_pnl.append(pnl)
        trade_ret.append((row.Close / last_price - 1) * 100)
        
        balance = balance + (row.Close * position)
        trade_balance.append(balance)
        
        pnl = 0
        position = 0
        last_signal = 'hold'

        c = 0

      # check signal and enter any possible position
      if row.LONG and last_signal != 'long':
        last_signal = 'long'
        last_price = row.Close
        trade_date_start.append(row.name)
        trade_side.append('long')

        position = int(balance / row.Close)
        cost = position * row.Close
        balance = balance - cost
        c = 0

      if row.SHORT and last_signal != 'short':
        last_signal = 'hold'
        
        c = 0
      
      if stop_loss_lvl:
        # check stop loss
        if position > 0 and ((row.Low / last_price)- 1) * 100 <= stop_loss_lvl and row.LONG == False:
          c = c + 1

          trade_date_end.append(row.name)
          trade_days.append(c)

          stop_loss_price = last_price + round(last_price * (stop_loss_lvl / 100), 4)

          pnl = (stop_loss_price - last_price) * position
          trade_pnl.append(pnl)
          trade_ret.append(((stop_loss_price / last_price) - 1) * 100)
          balance = balance + (stop_loss_price * position)
          trade_balance.append(balance)
          
          pnl = 0
          position = 0
          last_signal = 'hold'

          c = 0
    
      # compute market value and count days for any possible poisition
      if last_signal == 'hold':
        market_value = balance
      elif last_signal == 'long':
        c = c + 1
        market_value = (position * row.Close) + balance
      else: 
        c = c + 1
        #market_value = (row.Close - last_price) * position + balance
        #market_value = balance
      
      cum_value.append(market_value)


  # generate analysis
  # performance over time
  cum_ret_df = pd.DataFrame(cum_value, index=bt_df.index, columns=['CUM_RET'])
  cum_ret_df['CUM_RET'] = (market_value / init_balance - 1) * 100
  cum_ret_df['BUY_HOLD'] = (bt_df.Close / bt_df.Open.iloc[0] - 1) * 100
  cum_ret_df['ZERO'] = 0
  cum_ret_df['Market_Value'] = market_value

  # trade stats
  size = min(len(trade_date_start), len(trade_date_end))

  tarde_dict = {
      'START': trade_date_start[:size],
      'END': trade_date_end[:size],
      'SIDE': trade_side[:size],
      'DAYS': trade_days[:size],
      'PNL': trade_pnl[:size],
      'RET': trade_ret[:size],
      'BAL': trade_balance[:size]
  }

  trade_df = pd.DataFrame(tarde_dict)

  num_trades = trade_df.groupby('SIDE').count()[['START']]
  num_trades_win = trade_df[trade_df.PNL > 0].groupby('SIDE').count()[['START']]

  avg_days = trade_df.groupby('SIDE').mean()[['DAYS']]

  avg_ret = trade_df.groupby('SIDE').mean()[['RET']]
  avg_ret_win = trade_df[trade_df.PNL > 0].groupby('SIDE').mean()[['RET']]
  avg_ret_loss = trade_df[trade_df.PNL < 0].groupby('SIDE').mean()[['RET']]

  std_ret = trade_df.groupby('SIDE').std()[['RET']]

  detail_df = pd.concat([
                        num_trades, num_trades_win, avg_days,
                        avg_ret, avg_ret_win, avg_ret_loss, std_ret
                        ], axis=1, sort=False)

  detail_df.columns = [
                      'NUM_TRADES', 'NUM_TRADES_WIN', 'AVG_DAYS', 
                      'AVG_RET', 'AVG_RET_WIN', 'AVG_RET_LOSS', 'STD_RET'
                      ]

  detail_df.round(2)

  # max drawdown
  mv_df = pd.DataFrame(cum_value, index=bt_df.index, columns=['MV'])

  days = len(mv_df)

  roll_max = mv_df.MV.rolling(window=days, min_periods=1).max()
  drawdown_val = mv_df.MV - roll_max
  drawdown_pct = (mv_df.MV / roll_max - 1) * 100

  # return all stats
  return {
      'cum_ret_df': cum_ret_df,
      'max_drawdown': {
          'value': round(drawdown_val.min(), 0), 
          'pct': round(drawdown_pct.min(), 2)
          },
      'trade_stats': detail_df,
      'trade_details': trade_df
  }

In [ ]:
from pandas import ExcelWriter

writer = ExcelWriter("/content/drive/My Drive/Data/Parameters/Strategies:" + str(end_date) + ".xlsx")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

etflist = ['sdow','udow','edz','edc','jpnl','bzq','brzu','fngd','fngu','yang','yinn','dust','nugt','jdst','jnug','gll','ugl','tza','tna','sqqq','tqqq','russ','rusl','drv','drn','zsl','agq',
  'soxs','soxl','spxs','spxl','tmv','tmf','kold','boil','svxy','uvxy','labd','labu','bis','bib','ery','erx','faz','fas','tecs','tecl']

#etflist = ['nrgu','nrgd'] #Start Date 3/1/2020 with 20 day buffer



for elt_id, elt in enumerate(etflist):
  ticker = elt
  writer_single = ExcelWriter("/content/drive/My Drive/Data/Parameters_" + ticker + "_" + str(end_date) + ".xlsx")

  #Get the OHLC and Volume
  df = yf.download(ticker, start=start_date_buffer, end=end_date)
  df.head()
  df = get_stock_backtest_data(ticker, start_date, end_date)
  df.head()

  #Set up Backtesting Interval
  bt_df = df[(df.index >= start_date) & (df.index <= end_date)]
  bt_df.head()


  #Try Multiple Strategies
  from itertools import product

  a = [5, 10]
  b = [1, 3]
  c = [2, 4]

  list(product(a, b, c))

  param_list = [a, b, c]

  list(product(*param_list))
  def test_func(**kwargs):
    a = kwargs.get('a', 10)
    b = kwargs.get('b', 2)
    c = kwargs.get('c', 2)

    print(a, b, c)


  test_func(a=1, b=2, c=3)
  param_dict = {'a': 1, 'b': 2, 'c': 3}

  test_func(**param_dict)
  param_name = ['a', 'b', 'c']
  param = [1, 2, 3]

  dict(zip(param_name, param))
  param_name = ['a', 'b', 'c']
  param = [1, 2, 3]

  dict(zip(param_name, param))
  dict(zip(['a', 'b', 'c'], [1, 2, 3]))
  param_name = ['a', 'b', 'c']
  param = [1, 2, 3]

  dict(zip(param_name, param))
  dict(zip(['a', 'b', 'c'], [1, 2, 3]))
  a = [5, 10]
  b = [1, 3]
  c = [2, 4]

  param_list = [a, b, c]

  param_name = ['a', 'b', 'c']

  param_dict_list = [dict(zip(param_name, param)) for param in list(product(*param_list))]

  param_dict_list
  for param_dict in param_dict_list:
    test_func(**param_dict)
  strategies = [
    # {
    #   'func': strategy_KeltnerChannel_origin,
    #   'param': {
    #     'n': [i for i in range(2, 30, 1)]
    #   }
    # },

    {
      'func': strategy_BollingerBands,
      'param': {
        'n': [i for i in range(2, 35, 1)],
        'n_rng': [1, 2]
      }
    },

    # {
    #   'func': strategy_MA,
    #   'param': {
    #     'n': [i for i in range(2, 35, 1)],
    #     'ma_type': ['sma', 'ema']
    #   }
    # },

    {
      'func': strategy_MACD,
      'param': {
        'n_slow': [i for i in range(2, 15)],
        'n_fast': [i for i in range(2, 15)],
        'n_sign': [i for i in range(2, 15)]
      }
    },

    # {
    #   'func': strategy_MACrossover,
    #   'param': {
    #     'n_slow': [i for i in range(2, 6)],
    #     'n_fast': [i for i in range(2, 8)],
    #     'n_middle': [i for i in range(2, 4)]
    #   }
    # },

    # {
    #   'func': strategy_RSI,
    #   'param': {
    #     'n': [i for i in range(2, 35)]
    #   }
    # },

    # {
    #   'func': strategy_WR,
    #   'param': {
    #     'n': [i for i in range(2, 35)]
    #   }
    # },

    # {
    #   'func': strategy_Stochastic_fast,
    #   'param': {
    #     'k': [i for i in range(2, 26)],
    #     'd': [i for i in range(2, 26)]
    #   }
    # },

    # {
    #   'func': strategy_Stochastic_slow,
    #   'param': {
    #     'k': [i for i in range(2, 15)],
    #     'd': [i for i in range(2, 15)],
    #     'dd': [i for i in range(1, 10)]
    #   }
    # },

    # {
    #   'func': strategy_Ichmoku,
    #   'param': {
    #     'n_conv': [i for i in range(2, 15)],
    #     'n_base': [i for i in range(2, 15)],
    #     'n_span_b': [26]
    #   }
    # },
  ]
  for s in strategies:
    func = s['func']
    param = s['param']

    param_name = []
    param_list = []

    for k in param:
      param_name.append(k)
      param_list.append(param[k])

    param_dict_list = [dict(zip(param_name, param)) for param in list(product(*param_list))]
    
    print(len(param_dict_list))
  #ticker = 'ISF.L'
  #start_date = '2019-01-01'
  #end_date = '2020-10-16'

  df = get_stock_backtest_data(ticker, start_date, end_date)

  stop_loss_lvl = [-i for i in range(2, 6, 1)]
  stop_loss_lvl.append(None)

  result_dict = {
      'strategy': [],
      'param': [],
      'stoploss': [],
      'return': [],
      'max_drawdown': []
  }
  for s in strategies:
    func = s['func']
    param = s['param']

    strategy_name = str(func).split(' ')[1]

    param_name = []
    param_list = []

    for k in param:
      param_name.append(k)
      param_list.append(param[k])

    param_dict_list = [dict(zip(param_name, param)) for param in list(product(*param_list))]
    total_param_dict = len(param_dict_list)

    c = 0

    for param_dict in param_dict_list:
      clear_output()
      c = c + 1
      print(ticker + ' Running backtest for {} - ({}/{})'.format(strategy_name, c, total_param_dict))

      for l in stop_loss_lvl:
        bt_df = prepare_stock_ta_backtest_data(
            df, start_date, end_date, 
            func, **param_dict)

        result = run_stock_ta_backtest(bt_df, stop_loss_lvl=l)

        result_dict['strategy'].append(strategy_name)
        result_dict['param'].append(str(param_dict))
        result_dict['stoploss'].append(l)
        result_dict['return'].append(result['cum_ret_df'].iloc[-1, 0])
        result_dict['max_drawdown'].append(result['max_drawdown']['pct'])


  df = pd.DataFrame(result_dict)
  dfsorted = df.sort_values('return', ascending=False).head(20)

  dfsorted.to_excel(writer, ticker)
  writer.save() 
  dfsorted.to_excel(writer_single, ticker)
  writer_single.save()
  toc = time.time()
  print("Minutes taken = " + str((toc-tic)/60.0))

In [ ]:
toc = time.time()
print("Minutes taken = " + str((toc-tic)/60.0))